In [42]:
import requests
import pandas as pd

def get_product_detail(pid):
    url = 'https://api.bunjang.co.kr/api/pms/v1/products-detail/{}?viewerUid=-1'.format(pid)
    response = requests.get(url) #url 요청 보내고 응답 받아옴
    data = response.json().get('data', {}).get('product', {})
    return {
            'pid': data.get('pid', ''), # 물품 id
            'name': data.get('name', ''), # 물품 이름
            'description': data.get('description', ''), #물품 설명
            'price': data.get('price', ''), #물품 이름
            'qty': data.get('qty', ''), # 물품 수량
            'saleStatus': data.get('saleStatus', ''), #물품 판매 상태
            'status': data.get('status', ''), #물품 상태 (새거,사용됨) 2개로 나눠져 있는거 같음
            'imageUrl': data.get('imageUrl', ''), #이미지 url 나중에 사용할 거 생각해서 받아옴
            'keywords' : data.get('keywords',''), # 등록자가 등록한 keywords
            'address' : data.get('address',''), # 등록자가 주소를 입력할때가 있는듯
            'label' : data.get('label','') 
        }

def bunjang(key_word, pages, items_number):
    """
    items_number 한 페이지당 가지고 오는 갯수인데 기본값은 96이였습니다!
    """
    data = []
    for page in range(pages):
        url = 'https://api.bunjang.co.kr/api/1/find_v2.json?order=date&n={}&page={}&req_ref=search&q={}&stat_device=w&stat_category_required=1&version=4'.format(items_number,page,key_word)
        response = requests.get(url)
        datas = response.json()['list']
        for piddata in datas:
            pid = piddata['pid']
            detail_data = get_product_detail(pid)
            if detail_data is not None:
                data.append(detail_data)
    
    df = pd.DataFrame(data)

    df['link'] = 'https://m.bunjang.co.kr/products/' + df['pid'].astype(str)
    df['market'] = '번개장터'
    df['keyword'] = key_word

    df.to_csv('bunjang_data.csv', index=False, encoding='utf-8-sig') # encoding='utf-8'로 하니깐 계속 파일이 깨져가지고 'utf-8(bom)'으로 지정해놓은것입니다!!



In [44]:
bunjang('노트북',1,30)